Descarga de módulos AMPL

In [1]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],  # modules to install
    license_uuid="bb2c71f6-2b4c-4570-8eaf-4db0b0d7349a", )# su UUID de licencia


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\gabop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Licensed to Bundle #6733.7184 expiring 20241231: ICI5142-01 INVESTIGACION DE OPERACIONES AVANZADAS (Advanced Operations Research), Prof. Guillermo Cabrera-Guerrero, Pontificia Universidad Catolica de Valparaiso.


In [2]:
%%writefile sscflp.mod
# Definición de parámetros
param n; # Número de clientes
param m; # Número de centros de distribución
param c{i in 1..n, j in 1..m}; # Costo de transporte del cliente i al centro j
param f{j in 1..m}; # Costo fijo de apertura del centro j
param d{i in 1..n}; # Demanda del cliente i
param b{j in 1..m}; # Capacidad del centro j

# Definición de variables de decisión
var x{j in 1..m} binary; # Si el centro j está abierto
var y{i in 1..n, j in 1..m} binary; # Si el cliente i es atendido por el centro j

# Función objetivo: Minimizar los costos totales
minimize TotalCost: sum{i in 1..n, j in 1..m} c[i,j] * y[i,j] + sum{j in 1..m} f[j] * x[j];

# Restricción: Cada cliente debe ser atendido por exactamente un centro
subject to AssignCliente{i in 1..n}: sum{j in 1..m} y[i,j] = 1;

# Restricción: La capacidad del centro no debe ser excedida
subject to Capacity{j in 1..m}: sum{i in 1..n} d[i] * y[i,j] <= b[j] * x[j];

# Restricción: Un cliente solo puede ser asignado a un centro si el centro está abierto
subject to OpenIfAssigned{i in 1..n, j in 1..m}: y[i,j] <= x[j];

Overwriting sscflp.mod


In [3]:
%%writefile mscflp.mod 
# Definición de parámetros
param m; # Número de centros de distribución
param n; # Número de clientes
param f{i in 1..m}; # Costo de apertura del centro i
param c{i in 1..m, j in 1..n}; # Costo de transporte del centro i al cliente j
param S{i in 1..m}; # Capacidad del centro i
param D{j in 1..n}; # Demanda del cliente j

# Definición de variables de decisión
var x{i in 1..m, j in 1..n} >= 0; # Cantidad de demanda del cliente j atendida por el centro i
var y{i in 1..m} binary; # Si el centro i está abierto

# Función objetivo: Minimizar los costos totales
minimize TotalCost: sum{i in 1..m} f[i] * y[i] + sum{i in 1..m, j in 1..n} c[i,j] * x[i,j];

# Restricción: La capacidad del centro no debe ser excedida
subject to Capacity{i in 1..m}: sum{j in 1..n} x[i,j] <= S[i];

# Restricción: La demanda de cada cliente debe ser satisfecha
subject to Demand{j in 1..n}: sum{i in 1..m} x[i,j] >= D[j];

# Restricción: Un centro solo puede satisfacer demanda si está abierto
subject to OpenIfAssigned{i in 1..m, j in 1..n}: x[i,j] <= S[i] * y[i];


Overwriting mscflp.mod


In [4]:
def leer_instancia(filepath):
    with open(filepath, 'r') as file:
        # Read the number of warehouses (m) and customers (n)
        m, n = map(int, file.readline().split())
        
        # Read capacities and fixed costs for each warehouse
        capacities = []
        fixed_costs = []
        for _ in range(m):
            capacity, fixed_cost = file.readline().split()
            capacities.append(int(capacity))
            fixed_costs.append(float(fixed_cost))
        
        # Read demands and allocation costs for each customer
        demands = []
        allocation_costs = []
        for _ in range(n):
            # Read the demand for each customer
            demand = int(file.readline().strip())
            demands.append(demand)
            
            # Read the allocation costs of this customer to each warehouse
            costs = list(map(float, file.readline().split()))
            if len(costs) != m:
                raise ValueError(f"Expected {m} allocation costs for customer {_+1}, but got {len(costs)}.")
            allocation_costs.append(costs)
    
    # Return the data in a structured dictionary
    return {
        'm': m,
        'n': n,
        'capacities': capacities,
        'fixed_costs': fixed_costs,
        'demands': demands,
        'allocation_costs': allocation_costs
    }

# Example usage to read and store data in a variable
datos = leer_instancia("cd/../Problem set OR Library/cap41.txt")


ValueError: Expected 16 allocation costs for customer 1, but got 7.

In [ ]:
def crear_sscflp_dat(datos, output_path="sscflp.dat"):
    with open(output_path, 'w') as sscflp_dat:
        sscflp_dat.write(f'param n := {datos["n"]};\nparam m := {datos["m"]};\n')
        
        # Parameter c[i,j] - Cost to allocate demand from customer i to warehouse j
        sscflp_dat.write('param c : ' + ' '.join(map(str, range(1, datos['m'] + 1))) + ' :=\n')
        for i in range(datos['n']):
            sscflp_dat.write(f"{i+1} " + ' '.join(map(str, datos['allocation_costs'][i])) + '\n')
        sscflp_dat.write(';\n')
        
        # Parameter f[j] - Fixed cost of opening each warehouse
        sscflp_dat.write('param f :=\n' + '\n'.join([f"{j+1} {datos['fixed_costs'][j]}" for j in range(datos['m'])]) + ';\n')
        
        # Parameter d[i] - Demand of each customer
        sscflp_dat.write('param d :=\n' + '\n'.join([f"{i+1} {datos['demands'][i]}" for i in range(datos['n'])]) + ';\n')
        
        # Parameter b[j] - Capacity of each warehouse
        sscflp_dat.write('param b :=\n' + '\n'.join([f"{j+1} {datos['capacities'][j]}" for j in range(datos['m'])]) + ';\n')

# Create sscflp.dat using the data read
crear_sscflp_dat(datos)


NameError: name 'datos' is not defined

In [ ]:
def crear_mscflp_dat(datos, output_path="mscflp.dat"):
    with open(output_path, 'w') as mscflp_dat:
        mscflp_dat.write(f'param m := {datos["m"]};\nparam n := {datos["n"]};\n')
        
        # Parameter f[i] - Fixed cost of opening each warehouse
        mscflp_dat.write('param f :=\n' + '\n'.join([f"{i+1} {datos['fixed_costs'][i]}" for i in range(datos['m'])]) + ';\n')
        
        # Parameter c[i,j] - Allocation cost from each warehouse i to customer j
        mscflp_dat.write('param c : ' + ' '.join(map(str, range(1, datos['n'] + 1))) + ' :=\n')
        for i in range(datos['m']):
            mscflp_dat.write(f"{i+1} " + ' '.join(map(str, [datos['allocation_costs'][j][i] for j in range(datos['n'])])) + '\n')
        mscflp_dat.write(';\n')
        
        # Parameter S[i] - Capacity of each warehouse
        mscflp_dat.write('param S :=\n' + '\n'.join([f"{i+1} {datos['capacities'][i]}" for i in range(datos['m'])]) + ';\n')
        
        # Parameter D[j] - Demand of each customer
        mscflp_dat.write('param D :=\n' + '\n'.join([f"{j+1} {datos['demands'][j]}" for j in range(datos['n'])]) + ';\n')

# Create mscflp.dat using the data read
crear_mscflp_dat(datos)


In [ ]:
from amplpy import AMPL

def ejecutar_sscflp():
    ampl = AMPL()
    ampl.read('sscflp.mod')
    ampl.read('sscflp.dat')
    
    # Configurar solver
    ampl.setOption('solver', 'gurobi')
    ampl.solve()
    
    # Obtener y mostrar resultados
    resultado_x = ampl.var['x'].getValues().toPandas()
    resultado_y = ampl.var['y'].getValues().toPandas()
    print(resultado_x)
    print(resultado_y)

ejecutar_sscflp()


OSError: Can't find  file "sscflp.dat"
context:   >>> model 'sscflp.dat' <<< ; model;

In [ ]:
from amplpy import AMPL

def ejecutar_mscflp():
    ampl = AMPL()
    ampl.read('mscflp.mod')
    ampl.read('mscflp.dat')
    
    # Configurar solver
    ampl.setOption('solver', 'gurobi')
    ampl.solve()
    
    # Obtener y mostrar resultados
    resultado_x = ampl.var['x'].getValues().toPandas()
    resultado_y = ampl.var['y'].getValues().toPandas()
    print(resultado_x)
    print(resultado_y)

ejecutar_mscflp()
